In [1]:
import os
import pandas as pd
import h3
import random
from tqdm import tqdm
from collections import defaultdict
import pickle
import psutil
import time

In [2]:
file_path = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_train_PUNJAB_10.csv'

triplet_df = pd.read_csv(file_path)

In [3]:
triplet_df['cleaned_address'][2223]

'red apple living apartment room no 19 kapurthala phagwara law gate near to lovely professional university Phagwara'

In [4]:
def generate_triplets_from_csv(csv_path, resolution, K, T, D, output_pickle_path):
    df = pd.read_csv(csv_path)
    addresses = df['cleaned_address'].tolist()
    lat_lngs = df[['lookup_lat', 'lookup_lng']].values.tolist()

    def generate_h3_cells(lat_lngs, resolution):
        return [h3.geo_to_h3(lat, lng, resolution) for lat, lng in lat_lngs]

    h3_cells = generate_h3_cells(lat_lngs, resolution)

    def is_valid_h3_cell(cell):
        try:
            h3.h3_get_resolution(cell)
            return True
        except h3.H3CellError:
            return False

    def generate_triplets_on_the_fly(h3_cells, K, T, D, resolution):
        cell_to_points = defaultdict(list)
        h3_cells_set = set(h3_cells)

        for index, cell in enumerate(h3_cells):
            if is_valid_h3_cell(cell):
                cell_to_points[cell].append(index)

        triplets = []
        for cell, points in tqdm(cell_to_points.items(), desc="Generating Triplets", unit="cell"):
            if len(points) > 1:
                current_resolution = h3.h3_get_resolution(cell)
                if current_resolution >= resolution:
                    try:
                        parent_cell = h3.h3_to_parent(cell, resolution - 1)
                    except (h3.H3CellError, h3.H3ResolutionError):
                        continue

                    rings = []
                    for ring_level in range(1, D + 1):
                        if ring_level == 1:
                            ring = h3.k_ring(parent_cell, 2) - h3.k_ring(parent_cell, 1)
                        else:
                            previous_ring = h3.k_ring(parent_cell, ring_level)
                            ring = h3.k_ring(parent_cell, ring_level + 1) - previous_ring
                        rings.append(ring)

                valid_negative_samples_by_level = {ring_level: set() for ring_level in range(1, D + 1)}
                for ring_level, ring in enumerate(rings, start=1):
                    for neighbor in ring:
                        children = h3.h3_to_children(neighbor, resolution)
                        random.shuffle(list(children))
                        for child in children:
                            if child in h3_cells_set:
                                valid_negative_samples_by_level[ring_level].add(child)
                                if len(valid_negative_samples_by_level[ring_level]) >= 10000:
                                    break
                        if len(valid_negative_samples_by_level[ring_level]) >= 10000:
                            break

                for point in points:
                    positive_points = [p for p in points if p != point]
                    if positive_points:
                        sampled_positives = random.sample(positive_points, min(K, len(positive_points)))
                        for positive_sample in sampled_positives:
                            for ring_level in range(1, D + 1):
                                valid_negative_samples = valid_negative_samples_by_level[ring_level]
                                if valid_negative_samples:
                                    sampled_negatives = random.sample(list(valid_negative_samples), min(T, len(valid_negative_samples)))
                                    for neg in sampled_negatives:
                                        neg_index = h3_cells.index(neg)
                                        triplets.append((point, positive_sample, neg_index, ring_level))

        return triplets

    triplets = generate_triplets_on_the_fly(h3_cells, K, T, D, resolution)

    # Save the triplets, addresses, and lat_lngs into one pickle file
    with open(output_pickle_path, 'wb') as f:
        pickle.dump({
            'triplets': triplets,
            'addresses': addresses,
            'lat_lngs': lat_lngs
        }, f)

    print(f"Triplets saved to {output_pickle_path}")

In [5]:
csv_path = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_train_PUNJAB_10.csv'
resolution = 10
K = 1  
T = 1 
D = 3  
output_pickle_path = '/disk_4/uddeshya/Dataset/combined_triplets_PUNJAB_10.pkl'

In [6]:
generate_triplets_from_csv(csv_path, resolution, K, T, D, output_pickle_path)

Generating Triplets: 100%|██████████| 61664/61664 [37:33<00:00, 27.37cell/s]  


Triplets saved to /disk_4/uddeshya/Dataset/combined_triplets_PUNJAB_10.pkl


In [4]:
import pickle

# Path to the pickle file
input_pickle_path = '/disk_4/uddeshya/Dataset/combined_triplets_PUNJAB_05.1.pkl'

# Read the pickle file
with open(input_pickle_path, 'rb') as f:
    data = pickle.load(f)

# Get the lists from the pickle file
triplets = data.get('triplets', [])
addresses = data.get('addresses', [])
lat_lngs = data.get('lat_lngs', [])

# Get the length of each list
triplets_length = len(triplets)
addresses_length = len(addresses)
lat_lngs_length = len(lat_lngs)

# Print the lengths
print(f"Length of triplets: {triplets_length}")
print(f"Length of addresses: {addresses_length}")
print(f"Length of lat_lngs: {lat_lngs_length}")


Length of triplets: 3394661
Length of addresses: 1172900
Length of lat_lngs: 1172900


In [8]:
282737*3

848211

In [3]:
import pickle

# Path to the pickle file
input_pickle_path = '/disk_4/uddeshya/Dataset/combined_triplets_PUNJAB_05.1.pkl'

# Read the pickle file
with open(input_pickle_path, 'rb') as f:
    data = pickle.load(f)

# Get the lists from the pickle file
triplets = data.get('triplets', [])
addresses = data.get('addresses', [])
lat_lngs = data.get('lat_lngs', [])

# Get the length of each list
triplets_length = len(triplets)
addresses_length = len(addresses)
lat_lngs_length = len(lat_lngs)

# Print the lengths
print(f"Length of triplets: {triplets_length},{triplets[-1]}")
print(f"Length of addresses: {addresses_length},{addresses[-1]}")
print(f"Length of lat_lngs: {lat_lngs_length},{lat_lngs[-1]}")

Length of triplets: 3394661,(1172878, 1172877, 1172860, 3)
Length of addresses: 1172900,wardno 1 vpo sarhala ranuan Shahid Bhagat Singh Nagar District
Length of lat_lngs: 1172900,[31.2044125, 75.8583784]


In [3]:
print(triplets[-1])

(1172878, 1172877, 1172860, 3)


In [4]:
print(addresses[-1])

wardno 1 vpo sarhala ranuan Shahid Bhagat Singh Nagar District


In [5]:
print(lat_lngs[-1])

[31.2044125, 75.8583784]


In [6]:
import h3
import folium

In [8]:
def generate_h3_cells(lat_lngs, resolution):
    return [h3.geo_to_h3(lat, lng, resolution) for lat, lng in lat_lngs]

resolution = 10
h3_cells = generate_h3_cells(lat_lngs, resolution)
len(h3_cells)

1172900

In [13]:
def add_hexagons_to_map(h3_cells, color, m):
    for cell in h3_cells:
        boundary = h3.h3_to_geo_boundary(cell)
        boundary = [(lat, lng) for lat, lng in boundary]
        folium.Polygon(
            locations=boundary,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.0003,
            weight=2
        ).add_to(m)

def visualize_triplets_with_grid(lat_lngs, triplets, h3_cells, resolution, D):

    average_lat = sum(lat for lat, lng in lat_lngs) / len(lat_lngs)
    average_lng = sum(lng for lat, lng in lat_lngs) / len(lat_lngs)
    m = folium.Map(location=[average_lat, average_lng], zoom_start=13)

    colors = ['orange', 'red', 'purple', 'blue']

    for triplet in triplets:

        anchor_idx, positive_idx, negative_idx, ring_level = triplet


        anchor_coord = lat_lngs[anchor_idx]
        positive_coord = lat_lngs[positive_idx]
        negative_coord = lat_lngs[negative_idx]

        folium.Marker(location=anchor_coord, popup='Anchor', icon=folium.Icon(color='blue')).add_to(m)
        folium.Marker(location=positive_coord, popup='Positive', icon=folium.Icon(color='green')).add_to(m)
        folium.Marker(location=negative_coord, popup='Negative', icon=folium.Icon(color='red')).add_to(m)

        folium.PolyLine(locations=[anchor_coord, positive_coord], color='red' , weight=1.5).add_to(m)
        folium.PolyLine(locations=[anchor_coord, negative_coord], color='blue', weight=1.5).add_to(m)

        anchor_cell = h3_cells[anchor_idx]
        add_hexagons_to_map([anchor_cell], 'blue', m)

        positive_cell = h3_cells[positive_idx]
        add_hexagons_to_map([positive_cell], 'green', m)

        parent_cell = h3.h3_to_parent(anchor_cell, resolution - 1)
        add_hexagons_to_map([parent_cell], 'yellow', m)

        rings = []
        for ring_level in range(1, D + 1):
            if ring_level == 1:
                ring = h3.k_ring(parent_cell, 1) - {parent_cell}
            else:
                previous_ring = h3.k_ring(parent_cell, ring_level - 1)
                ring = h3.k_ring(parent_cell, ring_level) - previous_ring
            rings.append(ring)

        for ring_level in range(1, D + 1):
            ring = rings[ring_level - 1]
            add_hexagons_to_map(ring, colors[ring_level - 1], m)

    m.save('extensive_punjab_triplets.html')
    return m

In [14]:
D=3

In [15]:

if triplets:
    visualize_triplets_with_grid(lat_lngs, triplets[300:500], h3_cells, resolution, D+1)